# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

#### Methodology
To solve this problem, I'm going to use a two different clustering K-Means algorythm to classify the different attacks. One of them is going to try to group the data in two sets, while the other will try to do it in three sets. The one that fits the data better will tell us how many hackers were involved. Is important to note that this is possible because we know that all the hackers are expected to attack a similar number of times, otherwise the K-Means algorythm wouldn't be useful since it wouldn't discern well any minoritary group.

#### Initialize session & imports

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('kmeans2').getOrCreate()

### Read and visualize the data

In [0]:
dataset = spark.read.csv('/FileStore/tables/hack_data.csv', header=True, inferSchema=True)

In [0]:
dataset.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

The problem to solve is to determine if there are two or three different groups of data, that is, determine if the data adjust better to a clustering in two groups or three groups. With that objective in mind, I'm going to perform a Kmeans analysis on the sample. 
I'm going to use all the data except the location, given that in the description of the problem it is said that it is most probably irrelevant.

### Vector Assembling

AS said, I'm going to make a vector containing the information of all the columns that will be used in the building of the model, that is all except location.

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
dataset.columns

Out[10]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
relevant_columns = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [0]:
assembler = VectorAssembler(inputCols=relevant_columns, outputCol='features')

In [0]:
final_data = assembler.transform(dataset)

### Scaling

Although the data is not differing in many orders of magnitude, I'm going to scale it to avoid biases because of the model giving more importance to certain columns.

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [0]:
scaler_model = scaler.fit(final_data)

In [0]:
final_data = scaler_model.transform(final_data)

In [0]:
final_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|      scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58

### Kmeans Analysis

In this section, I'm going to build the two models with 2 and 3 clusters.

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
kmeans_2 = KMeans(featuresCol='scaledFeatures', k=2, seed=77)
kmeans_3 = KMeans(featuresCol='scaledFeatures', k=3, seed=77)

In [0]:
model_2 = kmeans_2.fit(final_data)
model_3 = kmeans_3.fit(final_data)

### Evaluating the models

In this section, I will analyse the accuracy of the models to determine which one adjust better to the data.

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
preds_2 = model_2.transform(final_data)
preds_3 = model_3.transform(final_data)

In [0]:
evaluator = ClusteringEvaluator()
silhouette_2 = evaluator.evaluate(preds_2)
silhouette_3 = evaluator.evaluate(preds_3)
print("Silhouette for 2 clusters: " , silhouette_2)
print('Silhouette for 3 clusters: ', silhouette_3)

Silhouette for 2 clusters:  0.6683623593283755
Silhouette for 3 clusters:  0.3068084951287429


The model of 2 clusters gives a Silhouette score much closer to 1 than the 3 model, so we can conclude that it fits the data better.

As a conclusion to the problem, we can claim that the number of hackers involved in the attack was 2.

Besides, considering the initial assumption that if there were another perpetrator, they should have made the same number of attacks. I could see the following trend:

In [0]:
preds_2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [0]:
preds_3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



The 2-clusters model splits the events evenly, but the 3-clusters model splits one of this groups in two, which confirms that there are only two evenly distributed groups in the sample.